In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
import visualization_utils as vis_utils
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix,MeshcatVisualizer, StartMeshcat
import pydrake.symbolic as sym
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions, ScsSolver
from pydrake.all import PointCloud, MeshcatVisualizerParams, Role, HalfSpace, CoulombFriction, Box, Rgba
from pydrake.polynomial import Polynomial as PolynomialCommon
import time
from pydrake.all import AddFrameTriadIllustration


from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import HPolyhedron, Hyperellipsoid
from pydrake.geometry.optimization_dev import CspaceFreePath
from tqdm import tqdm

from itertools import combinations, permutations
from joblib import Parallel, delayed


In [4]:
from rrt import BiRRT, StraightLineCollisionChecker

In [5]:
import logging
drake_logger = logging.getLogger("drake")
drake_logger.setLevel(logging.DEBUG)

In [6]:
meshcat = StartMeshcat()

DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is not set.
DEBUG:drake:FindRunfile mechanism = RUNFILES_{MANIFEST_FILE,DIR}
DEBUG:drake:cwd = "/home/amice/Documents/coding_projects/drake/C_Iris_Examples"
DEBUG:drake:FindRunfile found by-manifest '/home/amice/Documents/coding_projects/drake/.drake-find_resource-sentinel' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/7dof_iiwas_rrt.runfiles/drake/.drake-find_resource-sentinel' (good)
DEBUG:drake:FindRunfile found by-manifest '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/geometry/meshcat.js' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/7dof_iiwas_rrt.runfiles/drake/geometry/meshcat.js' (good)
DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [7]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
model_file = FindResourceOrThrow("drake/C_Iris_Examples/assets/iiwa6/iiwa6_with_box_collision.sdf")
box_file = FindResourceOrThrow("drake/C_Iris_Examples/assets/shelves.sdf")


parser = Parser(plant)

height = 0.1
ground_box = Box(10,10,0.1)
ground_offset = RigidTransform(p=[0,0,-height/2])
friction = CoulombFriction(1,1)
ground_collision = plant.RegisterCollisionGeometry(
    plant.world_body(), ground_offset, ground_box, "ground_col", friction
)
# ground_visual = plant.RegisterVisualGeometry(
#     plant.world_body(), ground_offset, ground_box, "ground", [1,0,0,0]
# )


models = []
models.append(Parser(plant, scene_graph).AddModelFromFile(model_file))
models.append(Parser(plant, scene_graph).AddModelFromFile(box_file))

sp = 0
x_fac = 0.7
z_offset = 0.0
locs = [ [0,0,0], 
        [x_fac, 1.4*sp,z_offset],
        [x_fac,-1.4*sp,z_offset],
        [-x_fac,-1.4*sp,z_offset],
        [-x_fac,1.4,z_offset], 
        [0.0 ,0 , z_offset],
        [0.0 ,0 , z_offset]] 
idx = 0
for model in models:
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame(),
                     RigidTransform(locs[idx]))
    idx +=1
    
# plant.WeldFrames(plant.GetBodyByName("iiwa_link_7").body_frame(), 
#                  plant.GetBodyByName("iiwa_link_6").body_frame(), 
#                  RigidTransform())


plant.Finalize()


        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(plant.num_positions())

do_viz = True

meshcat.Delete()
visualizer_params = MeshcatVisualizerParams()
visualizer_params.role = Role.kIllustration #Role.kProximity
visualizer = MeshcatVisualizer.AddToBuilder(
    builder, scene_graph, meshcat, visualizer_params)
diagram = builder.Build()

AddFrameTriadIllustration(scene_graph = scene_graph,
                        body = plant.get_body(plant.GetBodyIndices(models[-1])[0]))


diagram_context = diagram.CreateDefaultContext()
plant_context = diagram.GetMutableSubsystemContext(plant, diagram_context)
scene_graph_context = diagram.GetMutableSubsystemContext(scene_graph, diagram_context)
query_port = scene_graph.get_query_output_port()
diagram.ForcedPublish(diagram_context)

vis_bundle = vis_utils.VisualizationBundle(
    diagram, diagram_context, plant, plant_context,
    Ratfk, meshcat, q_star
)

DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is not set.
DEBUG:drake:FindRunfile found by-manifest '/home/amice/Documents/coding_projects/drake/.drake-find_resource-sentinel' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/7dof_iiwas_rrt.runfiles/drake/.drake-find_resource-sentinel' (good)
DEBUG:drake:FindRunfile found by-manifest '/home/amice/Documents/coding_projects/drake/C_Iris_Examples/assets/iiwa6/iiwa6_with_box_collision.sdf' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/7dof_iiwas_rrt.runfiles/drake/C_Iris_Examples/assets/iiwa6/iiwa6_with_box_collision.sdf' (good)
DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is not set.
DEBUG:drake:FindRunfile found by-manifest '/home/amice/Documents/coding_projects/drake/.drake-find_resource-sentinel' (goo

In [8]:
plant.num_positions()

6

In [9]:
#compute limits in s-space
limits_s = []
lower_q = plant.GetPositionLowerLimits()
upper_q = plant.GetPositionUpperLimits()

lower_q[0] = -np.pi/2
upper_q[0] = np.pi/2

lower_q[-1] = 0
upper_q[-1] = 0

lower_q[2] = 0

for q in [lower_q,upper_q]:
    limits_s.append(Ratfk.ComputeSValue(np.array(q), q_star))
limits_s = np.array(limits_s)
print(limits_s)

[[ -1.          -1.7320606    0.          -1.7320606  -11.43007018
    0.        ]
 [  1.           1.7320606   11.43007018   1.7320606   11.43007018
    0.        ]]


In [10]:
from pydrake.all import InverseKinematics
ik = InverseKinematics(plant, diagram.GetMutableSubsystemContext(plant, diagram_context))
min_dist = 1e-5
collision_constraint = ik.AddMinimumDistanceConstraint(
            min_dist, 1e-5)
diagram_col_context = diagram.CreateDefaultContext()
plant_col_context = diagram.GetMutableSubsystemContext(plant, diagram_col_context)
scene_graph_col_context = diagram.GetMutableSubsystemContext(scene_graph, diagram_col_context)
def check_collision_q_by_ik(q, min_dist=1e-5):
    if np.all(q >= plant.GetPositionLowerLimits()) and \
            np.all(q <= plant.GetPositionUpperLimits()):
        return 1 - 1 * \
            float(collision_constraint.evaluator().CheckSatisfied(q, min_dist))
    else:
        return 1
def check_collision_s_by_ik(s, min_dist=1e-5):
    s = np.array(s)
    q = Ratfk.ComputeQValue(s, q_star)
    return check_collision_q_by_ik(q, min_dist)

def check_collision_q_by_query(q):
    if np.all(q >= plant.GetPositionLowerLimits()) and \
            np.all(q <= plant.GetPositionUpperLimits()):
        plant.SetPositions(plant_col_context, q)
        query_object = query_port.Eval(scene_graph_col_context)
        return 1 if query_object.HasCollisions() else 0
    else:
        return 1
    
def check_collision_s_by_query(s):
    s = np.array(s)
    q = Ratfk.ComputeQValue(s, q_star)
    return check_collision_q_by_query(q)

In [11]:
# prm_important_samples_q = np.array([
# #     np.zeros(plant.num_positions()),
#     [-0.47, -0.19, 0.34, -1.89, 0.68, 0.18, 0],
#     [-2.1, -1.4,  1.53, -1.9 , -2.77,  0.6 ,  0],
#     [2.22, -2.09, -1.61, -1.61, -2.87,  0.64,  0],
#     [0.39,  0.41, -0.17, -1.79,  1.13, -0.49,  0]
# ]) 


prm_important_samples_q = np.array([
    np.zeros(plant.num_positions()),
    [ 0.033,  0.306, -0.067, -0.894,  0.033,  0.006],
    [ 0.033,  0.706, -0.367, -0.894,  0.033,  0.506],
    [-0.267,  1.106,  1.633, -0.894,  0.433, -0.894],
    [-0.467,  1.806,  1.833, -0.694,  0.433, -0.894],
    [-1.367,  1.606, -1.667,  2.094, -0.867, -0.094]
]) 


prm_important_samples_s = [ Ratfk.ComputeSValue(x, q_star) for x in prm_important_samples_q]

for s in prm_important_samples_s:
    print(check_collision_s_by_query(s))

0
0
0
0
0
0


In [12]:
col_checker = StraightLineCollisionChecker(check_collision_s_by_ik, 100)
# trees = Parallel(n_jobs=2)(delayed(RRT)((r1, r2, limits_s[0], limits_s[1], col_checker, 100))
#                            for (r1, r2) in combinations(prm_important_samples_s,2))
trees =[]
np.random.seed(1)
# for i, (r1, r2) in enumerate(combinations(prm_important_samples_s,2)):
for i, r2 in enumerate(prm_important_samples_s[1:]):
    r1 = prm_important_samples_s[0]
    trees.append(BiRRT(tuple(r1), tuple(r2), limits_s[0], limits_s[1], col_checker))
    tree_connected = trees[-1].build_tree(max_iter = int(1e2))
    print(tree_connected)
    if tree_connected:
        trees[-1].draw_tree(vis_bundle, plant.GetBodyByName("iiwa_link_7"), prefix = f"bi_rrt/{i}")
        

  1%|██▊                                                                                                                                                                                                                                                                                | 1/100 [00:00<00:06, 14.41it/s]


True


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 37.07it/s]


False


  5%|█████████████▊                                                                                                                                                                                                                                                                     | 5/100 [00:00<00:06, 15.74it/s]


True


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 46.73it/s]


False


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 30.78it/s]

False


## Set up the sliders so we can move the plant around manually

In [ ]:
sliders = []
q = np.zeros(plant.num_positions())#prm_important_samples_q[-1].copy()
# print(check_collision_q_by_query(q))
plant.SetPositions(plant_context, q)
diagram.ForcedPublish(diagram_context)
for i in range(plant.num_positions()):
    sliders.append(widgets.FloatSlider(min=plant.GetPositionLowerLimits()[i],
                                       max=plant.GetPositionUpperLimits()[i], 
                                       value=q[i], description=f'q{i}'))
# sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

#np.zeros(plant.num_positions())
def handle_slider_change(change, idx):
    q[idx] = change['new']
    plant.SetPositions(plant_context, q)
    diagram.ForcedPublish(diagram_context)
#     print(check_collision_q_by_query(q))
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


# visualizer.jupyter_cell()

In [ ]:
print(repr(np.round(plant.GetPositions(plant_context),3)))